# Nigep example using MNIST dataset

In [19]:
! pip3 install nigep

Defaulting to user installation because normal site-packages is not writeable


In [20]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/nigep/*'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.nigep import Nigep
import keras.layers as layers
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers.legacy import Adam
from keras.datasets import cifar10
from keras.utils import to_categorical


### Load and preprocess MNIST dataset from Keras package

In [21]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Converting the pixels data to float type
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Standardizing (255 is the total number of pixels an image can have)
X_train = X_train / 255
X_test = X_test / 255

OUTPUT_DIMS = 10

# One hot encoding the target class (labels)
y_train = to_categorical(y_train, OUTPUT_DIMS)
y_test = to_categorical(y_test, OUTPUT_DIMS)




### Build and compile the MLP model

In [22]:
NUM_EPOCHS = 20

model = Sequential()

model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(OUTPUT_DIMS, activation='softmax'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_28 (Ba  (None, 32, 32, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_25 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_29 (Ba  (None, 32, 32, 32)        128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                      

### Execute Nigep to obtain the Generalization Profile results

In [23]:
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_delta=0.001, patience=5, verbose=1)
erl_stopping = EarlyStopping(patience=4, monitor='val_loss', verbose=1)
callbacks = [erl_stopping]


nigep = Nigep(
    execution_name='cifar10',
    x_data=X_train,
    y_data=y_train,
    model=model,
    batch_size=64,
    input_shape=(32,32),
    class_mode='categorical',
    k_fold_n=5,
    target_names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
    epochs=20,
    callbacks=callbacks
)

nigep.execute()

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
nigep.plot_and_save_generalization_profile()